In [1]:
import ast
import datetime
import json
import os
import time

from elasticsearch import Elasticsearch
import pandas as pd
import pytrec_eval
from tqdm import tqdm

current_timestamp = datetime.datetime.now()

In [2]:
# Connect to local elastic

es = Elasticsearch('http://localhost:9200')
es.ping()

True

# Load Data

In [3]:
def convert_to_dict(string):
    try:
        return ast.literal_eval(string)
    except (SyntaxError, ValueError):
        return None  # Handle cases where the string is not a valid dictionary representation


## Products

In [4]:
# Products

filename = "processed_data/df_prods.csv"

df_prods = pd.read_csv(filename)
df_prods['product_attributes'] = df_prods['product_attributes'].apply(convert_to_dict)

df_prods.head()

,product_uid,product_title,product_description,product_attributes
0,100001,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",{'Bullet01': 'Versatile connector for various ...
1,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,BEHR Premium Textured DECKOVER is an innovativ...,"{'Application Method': 'Brush,Roller,Spray', '..."
2,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,Update your bathroom with the Delta Vero Singl...,"{'Bath Faucet Type': 'Combo Tub and Shower', '..."
3,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,Achieving delicious results is almost effortle...,"{'Appliance Type': 'Over the Range Microwave',..."
4,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,The Quantum Adjustable 2-Light LED Black Emerg...,"{'Battery Power Type': 'Ni-Cad', 'Battery Size..."


## Queries

In [5]:
# Queries

filename = "processed_data/df_queries.csv"

df_queries = pd.read_csv(filename)
df_queries.head()

,search_term,query_id,has_relevant_results
0,angle bracket,6e0a07626e48aee6f7ce9ec6cd753426d6acafded1598f...,1
1,l bracket,5863e75dfdc9ae5db3f6b4dbddf129d5568e085bf57711...,1
2,deck over,406b3569b2db043604fdb42a67f4ec49964a5ff07cddf0...,1
3,rain shower head,49b2dc56a0e1945c435c1579c07df519878619e3e8d59d...,1
4,shower only faucet,7620551bacb6cdddca5f33ec0943cea7971095a1e9be06...,1


## Query Relevance

In [6]:
# Join query_ids onto df_train

filename = "processed_data/df_relevance.csv"

df_relevance = pd.read_csv(filename)
df_relevance.head()

,query_id,product_uid,relevance
0,6e0a07626e48aee6f7ce9ec6cd753426d6acafded1598f...,100001,3.00
1,5863e75dfdc9ae5db3f6b4dbddf129d5568e085bf57711...,100001,2.50
2,406b3569b2db043604fdb42a67f4ec49964a5ff07cddf0...,100002,3.00
3,49b2dc56a0e1945c435c1579c07df519878619e3e8d59d...,100005,2.33
4,7620551bacb6cdddca5f33ec0943cea7971095a1e9be06...,100005,2.67


# Load into index

In [ ]:
# Initialize the index

index_name = 'products'

mapping = {
    "properties": {
        "product_uid": {
            "type": "integer"
        },
        "product_title": {
            "type": "text"
        },
        "product_description": {
            "type": "text"
        },
        "product_attributes": {
            "type": "nested",
            "properties": {
                "name": {
                    "type": "text"
                },
                "value": {
                    "type": "text"
                },
                "name_value": {
                    "type": "text"
                },
            }
        },
        "query_scores": {
            "type": "nested",
            "properties": {
                "query_id": {
                    "type": "text"
                },
                "relevance": {
                    "type": "float"
                },
            }
        }
    }
}

#es.indices.create(index=index_name, mappings=mapping)

In [ ]:
# Create documents for indexing

product_document_list = []

for index, row in tqdm(df_prods.iterrows(), total=len(df_prods)):
    
    query_scores = []
    tmp_query = df_relevance[(df_relevance['product_uid']==row['product_uid'])]
    if len(tmp_query)>0:
        tmp_query = tmp_query.replace(pd.NA, '', regex=True)
        for index_q, row_q in tmp_query.iterrows():
            query_scores.append({'query_id': row_q['query_id'],
                                 'relevance': row_q['relevance']})

    product_attributes = []
    tmp_attr = row['product_attributes']
    if not pd.isnull(tmp_attr):
        for k in tmp_attr.keys():
            product_attributes.append({'name': k,
                                    'value': tmp_attr[k],
                                    'name_value': str(k) + ' ' + str(tmp_attr[k])})

    tmp_doc = {
        'product_uid': row['product_uid'],
        'product_title': row['product_title'],
        'product_description': row['product_description'],
        'product_attributes': product_attributes,
        'query_scores': query_scores
    }
    product_document_list.append(tmp_doc)

In [ ]:
# Index the documents

for p in tqdm(product_document_list):
    es.index(index=index_name, document=p)

# Run Text Queries

In [8]:
def query_elasticsearch_hybrid(
    es_client,
    index_name,
    search_text=None,
    search_vector=None,
    num_results=10,
    boost_values=None,
):
    """Queries Elasticsearch with search and optional query_id filter.

    Args:
        es_client: An Elasticsearch client object.
        index_name: The name of the index to query.
        search_text: The text to search for.
        search_vector: The vector to search with.
        num_results: The maximum number of results to return (default 10).
        boost_values: A dictionary containing the boost values for title, description, and attributes.

    Returns:
        The search results from Elasticsearch.
    """

    # Default boost values
    boost_values = boost_values or {
        "title_boost": 1,
        "description_boost": 1,
        "attributes_boost": 1,
        "vector_boost": 1,
    }

    should_clauses = []

    # Text match clauses:
    if search_text is not None:
        text_match_clauses = [
            {
                "match": {
                    "product_title": {
                        "query": search_text,
                        "boost": boost_values["title_boost"],
                    }
                }
            },
            {
                "match": {
                    "product_description": {
                        "query": search_text,
                        "boost": boost_values["description_boost"],
                    }
                }
            },
            {
                "nested": {
                    "path": "product_attributes",
                    "query": {
                        "match": {
                            "product_attributes.name_value": {
                                "query": search_text,
                                "boost": boost_values["attributes_boost"],
                            }
                        }
                    },
                }
            },
        ]

        should_clauses.extend(text_match_clauses)
    
    # Vector match clauses:
    if search_vector is not None:
        vetor_match_clause = {
                        "knn": {
                            "field": "product_vector",
                            "query_vector": search_vector,
                            "num_candidates": 50,
                            "boost": boost_values["vector_boost"],
                        }
                    }
        should_clauses.append(vetor_match_clause)

    query_body = {
        "size": num_results,
        "query": {
            "bool": {
                "should": should_clauses
            }
        },
    }

    results = es_client.search(index=index_name, body=query_body)
    return results


## Sample Query

In [9]:
# A sample query

search_text = df_queries['search_term'][0]

results = query_elasticsearch_hybrid(es, 'products', search_text=search_text)

hits = pd.DataFrame(results['hits']['hits'])

In [10]:

product_uids = []
product_titles = []
product_descriptions = []
product_attributes = []
query_id_list = []
relevances = []
for h in hits['_source']:
    product_uids.append(h['product_uid'])
    product_titles.append(h['product_title'])
    product_descriptions.append(h['product_description'])
    product_attributes.append(h['product_attributes'])
    
    if len(h['query_scores'])>0:
        query_id_list.append(h['query_scores'][0]['query_id'])
        relevances.append(h['query_scores'][0]['relevance'])
    else:
        query_id_list.append(None)
        relevances.append(None)

hits['product_uid'] = product_uids
hits['product_title'] = product_titles
hits['product_description'] = product_descriptions
hits['product_attribute'] = product_attributes
hits['query_id'] = query_id_list
hits['relevance'] = relevances

hits.head(3)


,_index,_id,_score,_source,product_uid,product_title,product_description,product_attribute,query_id,relevance
0,products,GA0vEY8BkfDieeoVm0_n,39.548923,"{'product_uid': 198519, 'product_title': 'Vera...",198519,Veranda White Vinyl Traditional Left/Right Ang...,The Traditional Vinyl White Left/Right Angle B...,"[{'name': 'Accessory type', 'value': 'Left/Rig...",45476ecfe43c98557ae68d2eec00a37d8f21c0fef3d913...,2.0
1,products,wA0yEY8BkfDieeoVyNZS,33.432453,"{'product_uid': 170398, 'product_title': 'Marq...",170398,Marquee Railing Black Right Multi-Angle Bracke...,"Designed with a beautiful hammered-metal look,...","[{'name': 'Accessory type', 'value': 'Bracket'...",None,NaN
2,products,Rg0yEY8BkfDieeoV_uDh,33.432453,"{'product_uid': 174846, 'product_title': 'Marq...",174846,Marquee Railing Black Left Multi-Angle Bracket...,"Designed with a beautiful hammered-metal look,...","[{'name': 'Accessory type', 'value': 'Bracket'...",None,NaN


In [11]:
# Create ground truth dictionary

filename = "query_runs/qrel.json"

if os.path.isfile(filename):
    with open(filename, "r") as file:
        qrel = json.load(file)
else:
    qrel = {}

    for query in tqdm(df_relevance['query_id'].unique()):

        query_docs = df_relevance[df_relevance['query_id']==query].sort_values('relevance', ascending=False).reset_index(drop=True)
        query_doc_dict = {}
        for index, row in query_docs.iterrows():
            query_doc_dict[str(row['product_uid'])] = int(round(row['relevance']))-1
        
        qrel[query] = query_doc_dict
    with open(filename, "w") as file:
        json.dump(qrel, file)

## Run the text queries

In [12]:
# Create query result dictionaries from text queries

filename = "query_runs/text.json"

if os.path.isfile(filename):
    with open(filename, "r") as file:
        run_u = json.load(file)
else:

    start_time = time.time()

    run_u = {}

    for index, row in tqdm(df_queries[df_queries['has_relevant_results']==1].iterrows(),
                        total=len(df_queries[df_queries['has_relevant_results']==1])):
        search_term = row['search_term']
        search_query_id = row['query_id']
        
        results = query_elasticsearch_hybrid(es, 'products', search_text=search_term)
        hits = pd.DataFrame(results['hits']['hits'])

        query_doc_dict = {}
        for index, row in hits.iterrows():
            query_doc_dict[str(row['_source']['product_uid'])] = row['_score']
        
        run_u[search_query_id] = query_doc_dict

    end_time = time.time()

    with open(filename, "w") as file:
        json.dump(run_u, file)

100%|██████████| 11795/11795 [01:47<00:00, 109.47it/s]


## Evaluate results

In [13]:
# Place to store results and initialize an evaluator

ranking_results = []
measures = {'map_cut_10', 'ndcg_cut_10', 'recip_rank'} 
evaluator = pytrec_eval.RelevanceEvaluator(qrel, measures)

In [14]:
# Evaluate text queries

results_df = pd.DataFrame(evaluator.evaluate(run_u))
results_dict = {}
for measure in measures:
    results_dict['mean '+measure] = results_df.loc[measure].mean()
results_df = pd.DataFrame(results_dict, index=[0])

results_df['run_name'] = 'textsearch'
results_df['run_time'] = end_time - start_time
results_df['run_timestamp'] = current_timestamp

ranking_results.append(results_df)

In [15]:
# Put the results into a dataframe and save

pd.concat(ranking_results)

,mean ndcg_cut_10,mean map_cut_10,mean recip_rank,run_name,run_time,run_timestamp
0,0.184085,0.120573,0.265219,textsearch,107.798031,2024-04-24 15:28:41.859735


# Tune Text Boosts

In [ ]:
boost_list = []

for t in [1,2,4]:
    for d in [1,2,4]:
        for a in [1,2,4]:
            boost_list.append({"title_boost": t,
                               "description_boost": d,
                               "attributes_boost": a})

In [ ]:
# Run text queries with boosts

measures = {'map_cut_10', 'ndcg_cut_10', 'recip_rank'} 
evaluator = pytrec_eval.RelevanceEvaluator(qrel, measures)

for boosts in tqdm(boost_list):

    run = {}

    for index, row in tqdm(df_queries[df_queries['has_relevant_results']==1].iterrows(),
                        total=len(df_queries[df_queries['has_relevant_results']==1])):
        search_term = row['search_term']
        search_query_id = row['query_id']
        
        results = query_elasticsearch_hybrid(es, 'products', search_term, query_id_filter=None, num_results=10, boost_values=boosts)
        hits = pd.DataFrame(results['hits']['hits'])

        query_doc_dict = {}
        for index, row in hits.iterrows():
            query_doc_dict[str(row['_source']['product_uid'])] = row['_score']
        
        run[search_query_id] = query_doc_dict

    results_df = pd.DataFrame(evaluator.evaluate(run))
    results_dict = {}
    for measure in measures:
        results_dict['mean '+measure] = results_df.loc[measure].mean()
    results_df = pd.DataFrame(results_dict, index=[0])

    boosts['mean map_cut_10'] = results_df['mean map_cut_10'].values[0]
    boosts['mean ndcg_cut_10'] = results_df['mean ndcg_cut_10'].values[0]
    boosts['mean recip_rank'] = results_df['mean recip_rank'].values[0]


In [ ]:
pd.DataFrame(boost_list).sort_values('mean ndcg_cut_10')

In [ ]:
for t in [6, 8]:
    for d in [1,2,4]:
        for a in [1,2,4]:
            boost_list.append({"title_boost": t,
                               "description_boost": d,
                               "attributes_boost": a})

In [ ]:
for boosts in tqdm(boost_list[27:]):

    run = {}

    for index, row in tqdm(df_queries[df_queries['has_relevant_results']==1].iterrows(),
                        total=len(df_queries[df_queries['has_relevant_results']==1])):
        search_term = row['search_term']
        search_query_id = row['query_id']
        
        results = query_elasticsearch_hybrid(es, 'products', search_term, query_id_filter=None, num_results=10, boost_values=boosts)
        hits = pd.DataFrame(results['hits']['hits'])

        query_doc_dict = {}
        for index, row in hits.iterrows():
            query_doc_dict[str(row['_source']['product_uid'])] = row['_score']
        
        run[search_query_id] = query_doc_dict

    results_df = pd.DataFrame(evaluator.evaluate(run))
    results_dict = {}
    for measure in measures:
        results_dict['mean '+measure] = results_df.loc[measure].mean()
    results_df = pd.DataFrame(results_dict, index=[0])

    boosts['mean map_cut_10'] = results_df['mean map_cut_10'].values[0]
    boosts['mean ndcg_cut_10'] = results_df['mean ndcg_cut_10'].values[0]
    boosts['mean recip_rank'] = results_df['mean recip_rank'].values[0]

In [ ]:
pd.DataFrame(boost_list).sort_values('mean ndcg_cut_10')

## Run the boosted text queries

In [16]:
boosts = {"title_boost": 8,
          "description_boost": 2,
          "attributes_boost": 1}

In [17]:
# Create query result dictionaries from text queries

filename = "query_runs/run_boosted.json"

if os.path.isfile(filename):
    with open(filename, "r") as file:
        run_b = json.load(file)
else:
    run_b = {}

    start_time = time.time()

    for index, row in tqdm(df_queries[df_queries['has_relevant_results']==1].iterrows(),
                        total=len(df_queries[df_queries['has_relevant_results']==1])):
        search_term = row['search_term']
        search_query_id = row['query_id']
        
        results = query_elasticsearch_hybrid(es, 'products', search_text=search_term, boost_values=boosts)
        hits = pd.DataFrame(results['hits']['hits'])

        query_doc_dict = {}
        for index, row in hits.iterrows():
            query_doc_dict[str(row['_source']['product_uid'])] = row['_score']
        
        run_b[search_query_id] = query_doc_dict

    end_time = time.time()

    with open(filename, "w") as file:
        json.dump(run_b, file)

100%|██████████| 11795/11795 [01:49<00:00, 107.84it/s]


In [18]:
# Evaluate text queries boosted

results_df = pd.DataFrame(evaluator.evaluate(run_b))
results_dict = {}
for measure in measures:
    results_dict['mean '+measure] = results_df.loc[measure].mean()
results_df = pd.DataFrame(results_dict, index=[0])

results_df['run_name'] = 'textsearch_boosted'
results_df['run_time'] = end_time - start_time
results_df['run_timestamp'] = current_timestamp

ranking_results.append(results_df)

In [19]:
# Put the results into a dataframe and save

ranking_results = pd.concat(ranking_results)
ranking_results.to_csv('query_runs/query_results.csv', index=False)

ranking_results

,mean ndcg_cut_10,mean map_cut_10,mean recip_rank,run_name,run_time,run_timestamp
0,0.184085,0.120573,0.265219,textsearch,107.798031,2024-04-24 15:28:41.859735
0,0.236859,0.161795,0.320755,textsearch_boosted,109.380949,2024-04-24 15:28:41.859735


# Run the Sample Query again

In [20]:
sample_query_id = '2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd459c3b4cb7c3f3d85c2'

# A sample query

search_text = df_queries[df_queries['query_id']==sample_query_id]['search_term'].values[0]
print(search_text)

results = query_elasticsearch_hybrid(es, 'products', search_text=search_text)

hits = pd.DataFrame(results['hits']['hits'])

carpet grass 10 feet


In [21]:

product_uids = []
product_titles = []
product_descriptions = []
product_attributes = []
query_id_list = []
relevances = []
for h in hits['_source']:
    product_uids.append(h['product_uid'])
    product_titles.append(h['product_title'])
    product_descriptions.append(h['product_description'])
    product_attributes.append(h['product_attributes'])
    
    if len(h['query_scores'])>0:
        query_id_list.append(h['query_scores'][0]['query_id'])
        relevances.append(h['query_scores'][0]['relevance'])
    else:
        query_id_list.append(None)
        relevances.append(None)

hits['product_uid'] = product_uids
hits['product_title'] = product_titles
hits['product_description'] = product_descriptions
hits['product_attribute'] = product_attributes
hits['query_id'] = query_id_list
hits['relevance'] = relevances

hits.head(10)


,_index,_id,_score,_source,product_uid,product_title,product_description,product_attribute,query_id,relevance
0,products,Mw40EY8BkfDieeoV0TIF,28.645502,"{'product_uid': 209354, 'product_title': 'Scot...",209354,Scotts 10 lb. Turf Builder EZ Bermuda Grass Seed,The Scotts 10 lb. EZ Seed for Bermuda Grass La...,"[{'name': 'Bullet01', 'value': 'Grows in tough...",None,NaN
1,products,aQ0yEY8BkfDieeoVdsj4,26.909157,"{'product_uid': 163405, 'product_title': 'Robe...",163405,Roberts 2.125 in. Carpet Cutter Replacement Bl...,The Roberts 10-388-3 Replacement Trimmer blade...,"[{'name': 'Blade Length (In.)', 'value': '2.12...",None,NaN
2,products,vg41EY8BkfDieeoVGj5V,26.744383,"{'product_uid': 212565, 'product_title': 'IMAG...",212565,IMAGE 32 oz. Ready-to-Spray Herbicide for St. ...,IMAGE for St. Augustine grass and Centipede gr...,"[{'name': 'Application Type', 'value': 'Hose E...",None,NaN
3,products,IAwrEY8BkfDieeoVY6Ko,26.309332,"{'product_uid': 110504, 'product_title': 'GREE...",110504,GREENLINE Pink Blend Artificial Grass Syntheti...,GREENLINE Pink - add a splash of color to your...,"[{'name': 'Bullet01', 'value': 'Bright, vibran...",c29c67231f2c05d8d6d275547806be0dd48b92bc8c46b7...,2.00
4,products,CgwtEY8BkfDieeoVhPuY,26.309332,"{'product_uid': 151195, 'product_title': 'GREE...",151195,GREENLINE Slate Grey Artificial Grass Syntheti...,GREENLINE Grey - add a splash of color to your...,"[{'name': 'Bullet01', 'value': 'Bright, vibran...",9b0197111d7e430e28ba9b8035d52358a0c1833abade58...,3.00
5,products,0Q0zEY8BkfDieeoVZfKP,26.309332,"{'product_uid': 183338, 'product_title': 'GREE...",183338,GREENLINE Caribbean Blue Artificial Grass Synt...,GREENLINE Blue - add a splash of color to your...,"[{'name': 'Bullet01', 'value': 'Bright, vibran...",None,NaN
6,products,fg40EY8BkfDieeoVDRDd,26.309332,"{'product_uid': 196687, 'product_title': 'GREE...",196687,GREENLINE Caribbean Blue Artificial Grass Synt...,GREENLINE Blue - add a splash of color to your...,"[{'name': 'Bullet01', 'value': 'Bright, vibran...",None,NaN
7,products,9Q41EY8BkfDieeoVdE3b,26.309332,"{'product_uid': 216460, 'product_title': 'GREE...",216460,GREENLINE Slate Grey Artificial Grass Syntheti...,GREENLINE Grey - add a splash of color to your...,"[{'name': 'Bullet01', 'value': 'Bright, vibran...",None,NaN
8,products,bw0uEY8BkfDieeoVFROJ,26.301285,"{'product_uid': 164125, 'product_title': 'GREE...",164125,GREENLINE Jade 50 7.5 ft. x 10 ft. Artificial ...,Jade is our best value artificial grass. All t...,"[{'name': 'Bullet01', 'value': 'Appearance and...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,2.33
9,products,0wwtEY8BkfDieeoVHepn,25.716066,"{'product_uid': 143219, 'product_title': 'GREE...",143219,GREENLINE Jade 50 15 ft. x Your Length Artific...,Jade is our best value artificial grass. All t...,"[{'name': 'Bullet01', 'value': 'Appearance and...",a3efcb30d8cdb147b5b8622cfbfb56cead04dbbe2934a8...,2.33


In [22]:
# Rerun with boosts
results = query_elasticsearch_hybrid(es, 'products', search_text=search_text, boost_values=boosts)

hits = pd.DataFrame(results['hits']['hits'])

In [23]:

product_uids = []
product_titles = []
product_descriptions = []
product_attributes = []
query_id_list = []
relevances = []
for h in hits['_source']:
    product_uids.append(h['product_uid'])
    product_titles.append(h['product_title'])
    product_descriptions.append(h['product_description'])
    product_attributes.append(h['product_attributes'])
    
    if len(h['query_scores'])>0:
        query_id_list.append(h['query_scores'][0]['query_id'])
        relevances.append(h['query_scores'][0]['relevance'])
    else:
        query_id_list.append(None)
        relevances.append(None)

hits['product_uid'] = product_uids
hits['product_title'] = product_titles
hits['product_description'] = product_descriptions
hits['product_attribute'] = product_attributes
hits['query_id'] = query_id_list
hits['relevance'] = relevances

hits.head(10)


,_index,_id,_score,_source,product_uid,product_title,product_description,product_attribute,query_id,relevance
0,products,bw0uEY8BkfDieeoVFROJ,118.972030,"{'product_uid': 164125, 'product_title': 'GREE...",164125,GREENLINE Jade 50 7.5 ft. x 10 ft. Artificial ...,Jade is our best value artificial grass. All t...,"[{'name': 'Bullet01', 'value': 'Appearance and...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,2.33
1,products,gQwqEY8BkfDieeoV_JSo,115.103874,"{'product_uid': 104985, 'product_title': 'GREE...",104985,GREENLINE Classic 54 Fescue 5 ft. x 10 ft. Art...,Classic 54 is the best value artificial grass ...,"[{'name': 'Bullet01', 'value': 'Appearance and...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,2.67
2,products,JwwtEY8BkfDieeoVK-2A,115.103874,"{'product_uid': 144349, 'product_title': 'GREE...",144349,GREENLINE Classic 54 Fescue 7.5 ft. x 10 ft. A...,Classic 54 is the best value artificial grass ...,"[{'name': 'Bullet01', 'value': 'Appearance and...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,3.00
3,products,nAwrEY8BkfDieeoVD5dw,112.753320,"{'product_uid': 106230, 'product_title': 'GREE...",106230,GREENLINE Pet/Sport 60 5 ft. x 10 ft. Artifici...,Sport/Pet 60 is the perfect solution for sport...,"[{'name': 'Bullet01', 'value': 'Gentle on paws...",a3efcb30d8cdb147b5b8622cfbfb56cead04dbbe2934a8...,2.00
4,products,7A0uEY8BkfDieeoVHhSK,112.525740,"{'product_uid': 164899, 'product_title': 'GREE...",164899,GREENLINE Classic Pro 82 Spring 7.5 ft. x 10 f...,Classic Pro 82 is the top of the line in artif...,"[{'name': 'Bullet01', 'value': 'Appearance and...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,2.00
5,products,BQ0uEY8BkfDieeoVyDCA,112.269850,"{'product_uid': 180045, 'product_title': 'GREE...",180045,GREENLINE Classic Premium 65 Spring 5 ft. x 10...,Classic Premium 65 is our most popular artific...,"[{'name': 'Bullet01', 'value': 'Appearance and...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,2.00
6,products,mw0uEY8BkfDieeoVyzB7,112.269850,"{'product_uid': 180378, 'product_title': 'GREE...",180378,GREENLINE Classic Premium 65 Fescue 7.5 ft. x ...,Classic Premium 65 is our most popular artific...,"[{'name': 'Bullet01', 'value': 'Appearance and...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,1.33
7,products,iQ0yEY8BkfDieeoVpdDD,112.269850,"{'product_uid': 167422, 'product_title': 'GREE...",167422,GREENLINE Classic Premium 65 Spring 7.5 ft. x ...,Classic Premium 65 is our most popular artific...,"[{'name': 'Bullet01', 'value': 'Appearance and...",None,NaN
8,products,DA0vEY8BkfDieeoVj00s,109.624620,"{'product_uid': 197262, 'product_title': 'Turf...",197262,Turf Evolutions Deluxe Indoor Outdoor Landscap...,Turf Evolutions TruGrass Gold provides an envi...,"[{'name': 'Bullet01', 'value': 'All components...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,3.00
9,products,zw0uEY8BkfDieeoVYh83,109.296684,"{'product_uid': 170738, 'product_title': 'Turf...",170738,Turf Evolutions Luxurious Indoor Outdoor Lands...,Turf Evolutions TruGrass Tan provides an envir...,"[{'name': 'Bullet01', 'value': 'All components...",2702727dfc1076c3bd8affd42c05322c65ef8bb91e8dd4...,3.00
